In [1]:
# A cross-functional team with one data scientist working on an ML model

MLflow setup :
    - tracking server : yes, local server 
    - backend store : sqlite database 
    - artifacts store : local filesystem
    
The experiments can be explored locally by accessing the local tracking server. 

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

"mlflow server --backend-store-uri sqlite:///backend.db

In [2]:
import mlflow 

mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [3]:
print(f"tracking URI : '{mlflow.get_tracking_uri()}'")

tracking URI : 'http://127.0.0.1:5000'


In [4]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1758622014073, experiment_id='0', last_update_time=1758622014073, lifecycle_stage='active', name='Default', tags={}>]

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

In [6]:
mlflow.set_experiment("my-experiment-1")

2025/09/23 10:15:49 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


<Experiment: artifact_location='/workspaces/MLops-cookin-/02-MLflow/Practice/Scenario2/artifacts_local/1', creation_time=1758622549048, experiment_id='1', last_update_time=1758622549048, lifecycle_stage='active', name='my-experiment-1', tags={}>

In [10]:
with mlflow.start_run():
    X,y =load_iris(return_X_y=True)
    
    params= {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)
    
    lr = LogisticRegression(**params).fit(X, y)
    y_pred=lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))
    
    mlflow.sklearn.log_model(lr, name="models")
    print(f"default artifacts URI :'{mlflow.get_artifact_uri()}'")

2025/09/23 10:20:19 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpltt6ql9r/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 
2025/09/23 10:20:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI :'/workspaces/MLops-cookin-/02-MLflow/Practice/Scenario2/artifacts_local/1/1fd6179d182642a2999ace38a0b25db6/artifacts'
🏃 View run merciful-donkey-116 at: http://127.0.0.1:5000/#/experiments/1/runs/1fd6179d182642a2999ace38a0b25db6
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [11]:
mlflow.search_experiments()

[<Experiment: artifact_location='/workspaces/MLops-cookin-/02-MLflow/Practice/Scenario2/artifacts_local/1', creation_time=1758622549048, experiment_id='1', last_update_time=1758622549048, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1758622014073, experiment_id='0', last_update_time=1758622014073, lifecycle_stage='active', name='Default', tags={}>]

In [12]:
# Interacting with the model registry

In [15]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [17]:
client.search_registered_models()

[]

In [18]:
run_id= client.search_runs(experiment_ids="1")[0].info.run_id

In [19]:
run_id

'1fd6179d182642a2999ace38a0b25db6'

In [20]:
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name="iris_classifier"
)

Successfully registered model 'iris_classifier'.
2025/09/23 10:25:55 WARNING mlflow.tracking._model_registry.fluent: Run with id 1fd6179d182642a2999ace38a0b25db6 has no artifacts at artifact path 'models', registering model based on models:/m-6ba1b31123c04d6dbd578494e1cd2869 instead
2025/09/23 10:25:55 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris_classifier, version 1
Created version '1' of model 'iris_classifier'.


<ModelVersion: aliases=[], creation_timestamp=1758623155235, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1758623155235, metrics=None, model_id=None, name='iris_classifier', params=None, run_id='1fd6179d182642a2999ace38a0b25db6', run_link='', source='models:/m-6ba1b31123c04d6dbd578494e1cd2869', status='READY', status_message=None, tags={}, user_id='', version='1'>